In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# add your code here - consider creating a new cell for each section of code

In [ ]:
# Filter users with at least 200 ratings
user_counts = df_ratings['user'].value_counts()
df_ratings = df_ratings[df_ratings['user'].isin(user_counts[user_counts >= 200].index)]

# Filter books with at least 100 ratings
book_counts = df_ratings['isbn'].value_counts()
df_ratings = df_ratings[df_ratings['isbn'].isin(book_counts[book_counts >= 100].index)]

# Create pivot table with ISBN as index
book_user_matrix = df_ratings.pivot_table(index="isbn", columns="user", values="rating").fillna(0)

# Convert to sparse format
book_sparse_matrix = csr_matrix(book_user_matrix.values)

# Train KNN
model = NearestNeighbors(metric="cosine", algorithm="brute")
model.fit(book_sparse_matrix)

# Required function for grading
def get_recommends(book=""):
    # Find ISBN for given title
    isbn_list = df_books[df_books['title'] == book]['isbn'].values
    if len(isbn_list) == 0:
        return [book, []]
    isbn = isbn_list[0]

    if isbn not in book_user_matrix.index:
        return [book, []]

    idx = book_user_matrix.index.get_loc(isbn)
    distances, indices = model.kneighbors(book_sparse_matrix[idx], n_neighbors=6)

    recommendations = []
    for i in range(1, len(indices.flatten())):
        rec_isbn = book_user_matrix.index[indices.flatten()[i]]
        rec_title = df_books[df_books['isbn'] == rec_isbn]['title'].values[0]
        rec_distance = float(distances.flatten()[i])
        recommendations.append([rec_title, rec_distance])

    return [book, recommendations]


In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()